## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Aug 14 03:00:16 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,661569,35.4,1454232,77.7,1386331,74.1
Vcells,1231660,9.4,8388608,64.0,1975448,15.1


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




In [4]:
add_log = function(message){
    cat(message, file = paste0(getwd(), '/logs.txt'),append=TRUE)
}

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [5]:
PARAM <- list()
PARAM$semilla_primigenia <- 424243

PARAM$experimento <- '6300_gerencial_GENALGO_depth=20_gens=4_pop=1200'
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [6]:
# carpeta de trabajo
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(paste0("./exp/",experimento_folder))
setwd(paste0("./exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [7]:
# lectura del dataset
dataset <- fread("~/OneDrive/0.ITBA/Mineria_de_datos/dm2025a/src/workflows/gerencial_competencia_2025.csv.gz")

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [8]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [9]:
# sin codigo en esta primera version del workflow

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [10]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [11]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "mpayroll_sobre_edad"

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [12]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [13]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [14]:
ncol(dataset)
colnames(dataset)

[1] 158

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "kmes"                               "mpayroll_sobre_edad"               
 [35] "internet_lag1"                      "cliente_edad_lag1"                 
 [37] "cliente_antiguedad_lag1"            "mrentabilidad_lag1"                
 [39] "mrentabilidad_annual_lag1"          "mcomisiones_lag1"                  
 [41] "mactivos_margen_lag1"               "mpasivos_margen_lag1"              
 [43] "cproductos_lag1"                    "mcuenta_corriente_lag1"            
 [45] "mcaja_ahorro_lag1"                  "cdescubierto_preacordado_lag1"     
 [47] "mcuentas_saldo_lag1"                "ctarjeta_visa_transacciones_lag1"  
 [49] "mtarjeta_visa_consumo_lag1"         "mtarjeta_master_consumo_lag1"      
 [51] "mprestamos_personales_lag1"         "cpayroll_trx_lag1"                 
 [53] "mpayroll_lag1"                      "ccomisiones_mantenimiento_lag1"    
 [55] "ccallcenter_transacciones_lag1"     "chomebanking_transacciones_lag1"   
 [57] "ctrx_quarter_lag1"                  "Master_status_lag1"                
 [59] "Master_fechaalta_lag1"              "Master_mpagominimo_lag1"           
 [61] "Visa_status_lag1"                   "Visa_fechaalta_lag1"               
 [63] "Visa_mpagominimo_lag1"              "kmes_lag1"                         
 [65] "mpayroll_sobre_edad_lag1"           "internet_lag2"                     
 [67] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [69] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [71] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [73] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [75] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [77] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
 [79] "ctarjeta_visa_transacciones_lag2"   "mtarjeta_visa_consumo_lag2"        
 [81] "mtarjeta_master_consumo_lag2"       "mprestamos_personales_lag2"        
 [83] "cpayroll_trx_lag2"                  "mpayroll_lag2"                     
 [85] "ccomisiones_mantenimiento_lag2"     "ccallcenter_transacciones_lag2"    
 [87] "chomebanking_transacciones_lag2"    "ctrx_quarter_lag2"                 
 [89] "Master_status_lag2"                 "Master_fechaalta_lag2"             
 [91] "Master_mpagominimo_lag2"            "Visa_status_lag2"                  
 [93] "Visa_fechaalta_lag2"                "Visa_mpagominimo_lag2"             
 [95] "kmes_lag2"                          "mpayroll_sobre_edad_lag2"          
 [97] "internet_delta1"                    "internet_delta2"                   
 [99] "cliente_edad_delta1"                "cliente_edad_delta2"               
[1

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [15]:
# No se implementa la reduccion de la dimensionalidad con canaritos

In [16]:
#if( !require("rgenoud")) install.packages("rgenoud")
if( !require("gramEvol")) install.packages("gramEvol")
if( !require("primes")) install.packages("primes")
if( !require("lightgbm")) install.packages("lightgbm")
if( !require("GA")) install.packages("GA")
if( !require("stringr")) install.packages("stringr")

require("gramEvol")
require("primes")
require("GA")
require("stringr")
require("lightgbm")

Loading required package: gramEvol

Loading required package: primes

Loading required package: lightgbm

Loading required package: GA

Loading required package: foreach

Loading required package: iterators

Package 'GA' version 3.2.4
Type 'citation("GA")' for citing this R package in publications.


Attaching package: ‘GA’


The following object is masked from ‘package:utils’:

    de


Loading required package: stringr



In [17]:
campos_monetarios <- colnames(dataset)
campos_rf <- colnames(dataset)
campos_int <- colnames(dataset)
campos_lag <- colnames(dataset)

is_bool_column <- function(columname) {
    tryCatch({
      if (class(max(dataset[[columname]])) %in% c('numeric', 'integer')){
      maxx = max(dataset[[columname]])
      minn = min(dataset[[columname]])
      return ((maxx - minn) == 1)
} else{
          return (FALSE)
  }
}, error = function(e){
        FALSE
})}

get_non_bool_cols <- function(dataset) {
      all_cols <-ames(dataset)
      bool_cols <- sapply(all_cols, function(col) is_bool_column(dataset, col))
      return(all_cols[!bool_cols])
    }
                          
campos_a_omitir = c('numero_de_cliente','foto_mes','azar','clase_ternaria')
                          
actualizar_campos = function(){
    campos_monetarios <- colnames(dataset)
    campos_rf <- colnames(dataset)
    campos_int <- colnames(dataset)
    campos_lag <- colnames(dataset)
    campos_monetarios  <<- campos_monetarios[campos_monetarios %like%
      "^(m|Visa_m|Master_m|vm_m)"]
    
    campos_rf <<- campos_rf[campos_rf %like%
      "^rf"]
    campos_lag <<- campos_lag[campos_lag %like%
      "(lag|delta).$"]
    campos_lag <- campos_lag[!campos_lag %in% campos_monetarios]

    campos_int <<- campos_int[!campos_int %in% c(campos_a_omitir, campos_monetarios, campos_lag, campos_rf)]
    campos_int <<- campos_int[!sapply(campos_int,is_bool_column)]
    campos_int <<- na.omit(campos_int)
    }



In [18]:
safeEval <- function(expr) {
  val <- try(eval(parse(text = expr), envir = dataset), silent = TRUE)
  #if (inherits(val, "try-error") || any(is.nan(val)) || any(is.infinite(val))) {
  #  return(rep(NA, nrow(df)))
  #}
  val
}


In [19]:
# Grammar definition
actualizar_campos()

# Function to generate a new column from grammar
generate_column <- function() {
  #operators = c('-','+','/','*','^')
  expr_str = GrammarRandomExpression(grammarDef, startSymb = GrammarStartSymbol(grammarDef))
  #print(colnames(safeEval(parse(text=expr_str))))
  #while (str_count(colnames(safeEval(parse(text=expr_str))), operators) >= PARAM$GA$max_depth){
  #    expr_str <- GrammarRandomExpression(grammarDef, startSymb = GrammarStartSymbol(grammarDef))
  #}
  # Evaluate the generated expression
  val <- safeEval(expr_str)
  coln = gsub("dataset","",parse(text=expr_str))
  coln = gsub("\\[","",coln)
  coln = gsub("\\]","",coln)
  coln = gsub(", ","",coln)
  coln = gsub("\"","",coln)
  #print(coln)
  d = data.table(val)
  colnames(d) = c(coln)
  return(d)
}

In [20]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


In [21]:
campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosGeneticos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla,
  dataset) {

  cat( "inicio Canaritos Geneticos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01",'azar')
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]


  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin Canaritos Geneticos()\n")

  return( tb_importancia )
}


In [22]:
# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.2
PARAM$CN$desvios <- 2


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

In [23]:
PARAM$GA$generations <- 4
PARAM$GA$population <- 1200
PARAM$GA$max_depth <- 5
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$GA$generations)]


In [24]:



for (g in 1:PARAM$GA$generations){
    grammarDef <- CreateGrammar(list(
      expr = gsrule(
              "dataset[,<m_var>]/dataset[,<i_var>]",
              "dataset[,<m_var>]-dataset[,<m_var>]",
              "dataset[,<m_var>]/dataset[,<l_var>]",
              "dataset[,<m_var>]-dataset[,<l_var>]",
              "dataset[,<l_var>]/dataset[,<i_var>]",
              "dataset[,<l_var>]-dataset[,<l_var>]",
              "dataset[,<i_var>]/dataset[,<i_var>]",
              "dataset[,<i_var>]+dataset[,<i_var>]",
              "dataset[,<i_var>]-dataset[,<i_var>]"
      ),
      m_var  = gvrule(setdiff(campos_monetarios,campos_a_omitir)),
      i_var  = gvrule(setdiff(campos_int,campos_a_omitir)),
      l_var  = gvrule(setdiff(campos_lag,campos_a_omitir))
        )
   )
    set.seed(PARAM$FT$semillas[g])
    gen_dataset = copy(dataset[,c('numero_de_cliente','clase_ternaria','foto_mes')])
    for (i in 1:PARAM$GA$population) {
      new_col <- generate_column()
      gen_dataset = cbind(gen_dataset,new_col)
    }

    tb_importancia <- CanaritosGeneticos(
      canaritos_ratio = PARAM$CN$ratio,
      canaritos_desvios = PARAM$CN$desvios,
      canaritos_semilla = PARAM$semilla_primigenia,
      gen_dataset
    )
    fwrite( tb_importancia,
         file=paste0("canaritosGeneticos_",g,'.txt'),
         sep="\t"
        )
    dataset = cbind(dataset, gen_dataset)
    actualizar_campos()
}

head(dataset)

inicio CanaritosAsesinos()
fin CanaritosAsesinos()
inicio CanaritosAsesinos()
fin CanaritosAsesinos()
inicio CanaritosAsesinos()
fin CanaritosAsesinos()
inicio CanaritosAsesinos()
fin CanaritosAsesinos()


numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,mcomisiones_lag1/cliente_edad_lag2/kmes_lag2/mpayroll_lag2,mrentabilidad_delta1/mcuentas_saldo_delta1/mcuentas_saldo_lag1,Visa_mpagominimo/mpasivos_margen_lag1,mcuentas_saldo_delta1/mprestamos_personales_lag1/mpayroll_sobre_edad_lag2,mpayroll_lag1/cdescubierto_preacordado_lag1/mactivos_margen_lag2/mtarjeta_visa_consumo/cpayroll_trx_lag2/mcuentas_saldo_lag2/ccomisiones_mantenimiento_lag1,mcuenta_corriente_delta2/mcuenta_corriente_lag2/cproductos_delta1,mpayroll_sobre_edad/mactivos_margen_lag1/ccomisiones_mantenimiento/mcomisiones_lag2/Visa_fechaalta_lag1/mactivos_margen_lag2/ctarjeta_visa_transacciones_lag2/mcaja_ahorro_lag1/mpasivos_margen_lag2,mrentabilidad_lag1/mpayroll_sobre_edad_delta2/mcomisiones_lag2/ctarjeta_visa_transacciones_lag2,mcaja_ahorro/mcomisiones_delta1/mpayroll_sobre_edad/ctarjeta_visa_transacciones_lag1,mtarjeta_visa_consumo_lag1/mrentabilidad_annual_delta1/mcomisiones_delta1/mrentabilidad/cliente_edad_lag1/mactivos_margen_delta1
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,202005,1,61,282,2354.94,43761.59,1682.14,-139.02,712.60,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29187961,202005,1,61,312,5066.31,47268.33,3875.62,-3536.03,4551.11,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29193101,202005,1,67,365,4809.42,37996.66,6982.73,-2962.24,1109.46,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29193281,202005,1,54,158,2979.52,88436.59,3350.01,-908.55,592.70,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29198891,202005,1,54,312,3008.70,22567.32,273.68,-643.87,2975.52,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29205441,202005,1,72,6,394.19,1607.93,159.49,-9.92,210.01,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [25]:
# Esta es la parte que los alumnos deben desplegar todo su ingenio
# Agregar aqui sus PROPIAS VARIABLES manuales

limpiar_inf <- function(dataset_genetico) {
  cat( "inicio AgregarVariables_IntraMes()\n")
  add_log( "inicio AgregarVariables_IntraMes()\n")
  gc(verbose= FALSE)
  # INICIO de la seccion donde se deben hacer cambios con variables nuevas

  # valvula de seguridad para evitar valores infinitos
  # paso los infinitos a NULOS
  infinitos <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
        add_log(paste0(
      "ATENCION, hay ", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n")
    )

    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    add_log(
      paste0("ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    ))

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    add_log("Si no te gusta la decision, modifica a gusto el programa!\n\n")


    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat( "fin AgregarVariables_IntraMes()\n")
  add_log( "fin AgregarVariables_IntraMes()\n")
}
limpiar_inf(dataset)

inicio AgregarVariables_IntraMes()
ATENCION, hay 16445445 valores infinitos en tu dataset. Seran pasados a NA
ATENCION, hay 8954432 valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0
Si no te gusta la decision, modifica a gusto el programa!

fin AgregarVariables_IntraMes()


In [26]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


In [27]:
# Elimina del dataset las variables que estan por debajo
#  de la capa geologica de canaritos
# se llama varias veces, luego de agregar muchas variables nuevas,
#  para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1

campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosAsesinos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla) {

  cat( "inicio CanaritosAsesinos()\n")
  add_log( "inicio CanaritosAsesinos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01",'azar')
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]

  fwrite(tb_importancia,
    file = paste0("impo_", GVEZ, ".txt"),
    sep = "\t"
  )

  GVEZ <<- GVEZ + 1

  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin CanaritosAsesinos()\n")
  add_log( "fin CanaritosAsesinos()\n")

  return( tb_importancia )
}


In [28]:
# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.2
PARAM$CN$desvios <- 2


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

In [29]:
# la llamada a Canaritos Asesinos
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)


inicio CanaritosAsesinos()
fin CanaritosAsesinos()


In [30]:
# grabo la importancia, ver el archivo directamente en la carpeta

fwrite( tb_importancia,
 file="canaritos.txt",
 sep="\t"
)

In [31]:
tb_importancia


Feature,Gain,Cover,Frequency,pos
<chr>,<dbl>,<dbl>,<dbl>,<int>
ctrx_quarter/cproductos_lag2,0.140581373,0.011247010,0.006211180,1
ctrx_quarter,0.120692980,0.017947473,0.011293055,2
mpayroll_sobre_edad/ctarjeta_visa_transacciones_lag1/ccomisiones_mantenimiento,0.056720780,0.033767354,0.014680971,3
mtarjeta_visa_consumo,0.029478093,0.016812273,0.013551666,4
mcaja_ahorro/cliente_antiguedad,0.021068226,0.002118818,0.003952569,5
ctrx_quarter/cproductos_lag2/ccomisiones_mantenimiento,0.020379858,0.004781938,0.004517222,6
mpasivos_margen_lag2,0.015246011,0.004851224,0.003387916,7
mprestamos_personales/cliente_antiguedad_delta1,0.014761355,0.007691342,0.005081875,8
mcaja_ahorro,0.014257099,0.003208331,0.001693958,9


### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [32]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [33]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [34]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [35]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [36]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [37]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [38]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 44

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [39]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [40]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [41]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Thu Aug 14 03:32:05 PM 2025 AUC 0.927669094186696

Thu Aug 14 03:32:21 PM 2025 AUC 0.927610703949914

Thu Aug 14 03:32:46 PM 2025 AUC 0.931437581532002

Thu Aug 14 03:33:03 PM 2025 AUC 0.932919117936693

Thu Aug 14 03:33:27 PM 2025 AUC 0.927956411224828

Thu Aug 14 03:33:44 PM 2025 AUC 0.926225094362793

Thu Aug 14 03:33:59 PM 2025 AUC 0.926148630957484

Thu Aug 14 03:34:19 PM 2025 AUC 0.928119069741577

[mbo] 0: num_leaves=86; min_data_in_leaf=7015 : y = 0.928 : 17.5 secs : initdesign

[mbo] 0: num_leaves=182; min_data_in_leaf=5783 : y = 0.928 : 15.6 secs : initdesign

[mbo] 0: num_leaves=119; min_data_in_leaf=1320 : y = 0.931 : 25.0 secs : initdesign

[mbo] 0: num_leaves=62; min_data_in_leaf=416 : y = 0.933 : 16.8 secs : initdesign

[mbo] 0: num_leaves=155; min_data_in_leaf=3177 : y = 0.928 : 24.7 secs : initdesign

[mbo] 0: num_leaves=198; min_data_in_leaf=8111 : y = 0.926 : 16.4 secs : initdesign

[mbo] 0: num_leaves=20; min_data_in_

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [42]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:         68              273            204


### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [43]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [44]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [45]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [46]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [47]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [48]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [49]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [50]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [51]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "data-mining-gerencial-2025-a"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

In [52]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [53]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Aug 14 03:48:42 PM 2025"

In [54]:
NTFY_LINK <- "https://ntfy.sh/lh-dm2025"
notif <- function(mensaje){
        system(sprintf('curl -d  "%s -- %s" %s', format(Sys.time(), "%m-%d %H:%M"), mensaje, NTFY_LINK)) 
}
notif("GENALGO Test Done")